# Scraper de Trustpilot (Categoría: Viajes y Vacaciones)

Este notebook extrae información de empresas y reseñas de https://es.trustpilot.com/categories/travel_vacation usando Selenium.

## Variables que extrae:
- ID de la reseña, Dominio, Nombre de la empresa, Categorías, Subcategorías, Calificación de la empresa
- Fecha de la reseña, Nombre del cliente, Puntuación del cliente, Texto de la reseña
- Columnas vacías para LLM: Idioma, Sentimiento, Emoción, Género del cliente, Tema principal, Palabras clave, Tipo de cliente, Tipo de turista, Tipo de grupo, ¿Analizado?


In [1]:
# Importar librerías necesarias
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import re
from datetime import datetime
from tqdm import tqdm
import json
import hashlib


In [2]:
# Configuración del driver de Chrome
def setup_driver(headless=False):
    """Configura y retorna el driver de Chrome con las opciones necesarias"""
    chrome_options = Options()
    
    # Modo headless opcional
    if headless:
        chrome_options.add_argument('--headless')
    
    # Opciones para evitar detección
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument('--disable-blink-features=AutomationControlled')
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
    chrome_options.add_experimental_option('useAutomationExtension', False)
    
    # Desactivar imágenes para cargar más rápido (opcional)
    # prefs = {"profile.managed_default_content_settings.images": 2}
    # chrome_options.add_experimental_option("prefs", prefs)
    
    # User agent realista
    chrome_options.add_argument('user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36')
    
    # Tamaño de ventana
    chrome_options.add_argument('--window-size=1920,1080')
    
    # Otras opciones para parecer más humano
    chrome_options.add_argument('--disable-gpu')
    chrome_options.add_argument('--disable-extensions')
    chrome_options.add_argument('--proxy-server="direct://"')
    chrome_options.add_argument('--proxy-bypass-list=*')
    chrome_options.add_argument('--start-maximized')
    
    try:
        # Inicializar driver
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=chrome_options)
        
        # Ejecutar JavaScript para ocultar webdriver
        driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
        
        driver.implicitly_wait(10)
        print("✅ Driver Chrome iniciado correctamente")
        return driver
        
    except Exception as e:
        print(f"❌ Error al iniciar Chrome: {e}")
        raise e


In [3]:
# Función para delays aleatorios más humanos
import random

def random_delay(min_seconds=1, max_seconds=3):
    """Pausa aleatoria para parecer más humano"""
    delay = random.uniform(min_seconds, max_seconds)
    time.sleep(delay)


In [4]:
# Funciones auxiliares
def generate_review_id(company_name, review_date, customer_name, review_text):
    """Genera un ID único para cada reseña"""
    content = f"{company_name}{review_date}{customer_name}{review_text[:50]}"
    return hashlib.md5(content.encode()).hexdigest()[:12]

def parse_date(date_str):
    """Convierte la fecha del formato de Trustpilot a formato estándar"""
    try:
        # Mapeo de meses en español
        meses = {
            'enero': 'January', 'febrero': 'February', 'marzo': 'March',
            'abril': 'April', 'mayo': 'May', 'junio': 'June',
            'julio': 'July', 'agosto': 'August', 'septiembre': 'September',
            'octubre': 'October', 'noviembre': 'November', 'diciembre': 'December'
        }
        
        # Reemplazar mes español por inglés
        for mes_esp, mes_eng in meses.items():
            date_str = date_str.replace(mes_esp, mes_eng)
        
        # Parsear fecha
        return pd.to_datetime(date_str, format='%d de %B de %Y')
    except:
        return None

def scroll_to_load_reviews(driver, max_scrolls=10):
    """Hace scroll para cargar más reseñas"""
    last_height = driver.execute_script("return document.body.scrollHeight")
    scrolls = 0
    
    while scrolls < max_scrolls:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
            
        last_height = new_height
        scrolls += 1


In [5]:
# Función de debugging para analizar la estructura de la página
def debug_page_structure(driver, url):
    """Analiza la estructura de la página para encontrar selectores correctos"""
    print(f"\n🔍 DEBUGGING: Analizando estructura de {url}")
    driver.get(url)
    time.sleep(5)
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    # Buscar todos los enlaces que podrían ser de empresas
    review_links = soup.find_all('a', href=re.compile('/review/'))
    print(f"📌 Enlaces con /review/: {len(review_links)}")
    if review_links:
        print("   Primeros 3 ejemplos:")
        for i, link in enumerate(review_links[:3]):
            print(f"   - {link.get('href')} | Texto: {link.text.strip()[:50]}")
    
    # Buscar elementos article
    articles = soup.find_all('article')
    print(f"📌 Elementos <article>: {len(articles)}")
    
    # Buscar elementos con clases que contengan 'business' o 'company'
    business_elements = soup.find_all(class_=re.compile('business|company', re.I))
    print(f"📌 Elementos con clases 'business/company': {len(business_elements)}")
    
    # Buscar elementos con data attributes
    data_attrs = soup.find_all(attrs={'data-business-unit-card': True})
    print(f"📌 Elementos con data-business-unit-card: {len(data_attrs)}")
    
    # Guardar HTML para análisis manual
    with open('trustpilot_debug.html', 'w', encoding='utf-8') as f:
        f.write(str(soup.prettify()))
    print("💾 HTML guardado en trustpilot_debug.html")
    
    # Tomar screenshot
    driver.save_screenshot('trustpilot_debug.png')
    print("📸 Screenshot guardado en trustpilot_debug.png")
    
    return soup


In [6]:
# Función para extraer información de las empresas
def get_companies_from_category(driver, category_url, max_pages=75):
    """Extrae información de empresas de una categoría"""
    companies = []
    
    for page in range(1, max_pages + 1):
        url = f"{category_url}?page={page}"
        print(f"🔍 Accediendo a: {url}")
        driver.get(url)
        time.sleep(5)  # Aumentar tiempo de espera
        
        # Guardar HTML para debugging
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')
        
        # Diferentes selectores posibles para las tarjetas de empresas
        # ORDEN MODIFICADO: El selector 'paper_paper__' causaba duplicados (encontraba los mismos 10 elementos en cada página)
        selectors = [
            {'tag': 'div', 'attrs': {'data-business-unit-card': True}},
            {'tag': 'article', 'attrs': {'class': re.compile('styles_businessUnitCard')}},
            {'tag': 'div', 'attrs': {'class': re.compile('styles_businessUnitCard')}},
            # Usar directamente los enlaces que SÍ funcionan (debug muestra ~30 por página)
            {'tag': 'a', 'attrs': {'href': re.compile('/review/[^/?]+$')}},  # Solo enlaces que terminan con dominio
            # ELIMINADO: {'tag': 'div', 'attrs': {'class': re.compile('paper_paper__')}}, # ¡Este causaba los duplicados!
        ]
        
        company_cards = []
        for selector in selectors:
            company_cards = soup.find_all(selector['tag'], attrs=selector['attrs'])
            if company_cards:
                print(f"✅ Encontradas {len(company_cards)} tarjetas usando selector: {selector}")
                break
        
        if not company_cards:
            print(f"⚠️ No se encontraron tarjetas de empresas en la página {page}")
            print("🔍 Guardando screenshot para debugging...")
            driver.save_screenshot(f"trustpilot_page_{page}_debug.png")
            
            # Buscar enlaces alternativos
            all_links = soup.find_all('a', href=re.compile('/review/'))
            if all_links:
                print(f"📌 Encontrados {len(all_links)} enlaces de review")
            
            # Si es la primera página, intentar con wait más específico
            if page == 1:
                try:
                    print("⏳ Esperando carga completa de la página...")
                    WebDriverWait(driver, 15).until(
                        EC.presence_of_element_located((By.TAG_NAME, "article"))
                    )
                    soup = BeautifulSoup(driver.page_source, 'html.parser')
                    company_cards = soup.find_all('article')
                    if company_cards:
                        print(f"✅ Encontrados {len(company_cards)} articles después de esperar")
                except:
                    pass
            
            if not company_cards:
                continue
        
        # Si son enlaces directos, procesarlos de manera diferente
        if company_cards and company_cards[0].name == 'a':
            for link in company_cards:
                try:
                    company_url = link.get('href', '')
                    if '/review/' not in company_url:
                        continue
                        
                    if not company_url.startswith('http'):
                        company_url = f"https://es.trustpilot.com{company_url}"
                    
                    # Obtener dominio primero para limpiar el nombre
                    domain = company_url.split('/')[-1].split('?')[0]  # Remover parámetros
                    
                    # Obtener el nombre de la empresa con mejores estrategias
                    company_name = ""
                    
                    # Estrategia 1: Buscar en el texto del enlace y limpiar
                    link_text = link.get_text(separator=' ', strip=True)
                    if link_text:
                        # Limpiar texto común no deseado
                        company_name = link_text.replace('Más relevantes', '').replace('Más relevante', '').strip()
                        
                        # Si el dominio está en el texto, usar lo que está antes
                        if domain in company_name:
                            parts = company_name.split(domain)
                            if parts and parts[0]:
                                company_name = parts[0].strip()
                        
                        # Si hay números (rating), cortar antes del primer número
                        match = re.search(r'^([^0-9]+?)(?:\d|$)', company_name)
                        if match:
                            company_name = match.group(1).strip()
                    
                    # Estrategia 2: Buscar en elementos hijos si no encontramos nombre
                    if not company_name or len(company_name) < 2:
                        name_elem = link.find(['span', 'p', 'h2', 'h3'])
                        if name_elem:
                            company_name = name_elem.text.strip()
                    
                    # Fallback: usar el dominio limpio
                    if not company_name or len(company_name) < 2:
                        company_name = domain.replace('.com', '').replace('.es', '').replace('-', ' ').title()
                    
                    # Buscar información adicional en el contenedor padre
                    parent = link.parent
                    rating = "N/A"
                    num_reviews = "0"
                    
                    if parent:
                        # Buscar calificación
                        rating_elem = parent.find(text=re.compile(r'\d+[,\.]\d+'))
                        if rating_elem:
                            rating = re.search(r'\d+[,\.]\d+', rating_elem).group()
                        
                        # Buscar número de reseñas
                        reviews_elem = parent.find(text=re.compile(r'\d+\s*(reseñas?|reviews?|opiniones?)'))
                        if reviews_elem:
                            num_match = re.search(r'(\d+)', reviews_elem)
                            if num_match:
                                num_reviews = num_match.group(1)
                    
                    companies.append({
                        'company_name': company_name,
                        'domain': domain,
                        'company_url': company_url,
                        'rating': rating,
                        'num_reviews': num_reviews,
                        'categories': 'travel_vacation'
                    })
                    
                except Exception as e:
                    print(f"Error procesando enlace: {e}")
                    continue
        else:
            # Procesar tarjetas normales
            for card in company_cards:
                try:
                    # Buscar enlace principal
                    link_elem = card.find('a', href=re.compile('/review/'))
                    if not link_elem:
                        continue
                    
                    company_url = link_elem.get('href', '')
                    if not company_url.startswith('http'):
                        company_url = f"https://es.trustpilot.com{company_url}"
                    
                    # Nombre de la empresa - múltiples estrategias
                    company_name = None
                    name_selectors = [
                        ('p', {'class': re.compile('typography_heading')}),
                        ('span', {'class': re.compile('typography_heading')}),
                        ('h2', {}),
                        ('h3', {}),
                        ('p', {'class': re.compile('displayName')}),
                        ('a', {}),  # El mismo enlace
                    ]
                    
                    for tag, attrs in name_selectors:
                        name_elem = card.find(tag, attrs)
                        if name_elem and name_elem.text.strip():
                            company_name = name_elem.text.strip()
                            break
                    
                    if not company_name:
                        company_name = link_elem.text.strip() or company_url.split('/')[-1]
                    
                    # Dominio
                    domain = company_url.split('/')[-1] if company_url else "N/A"
                    
                    # Calificación
                    rating = "N/A"
                    rating_patterns = [r'\d+[,\.]\d+', r'\d+\.\d+', r'\d+,\d+']
                    for pattern in rating_patterns:
                        rating_elem = card.find(text=re.compile(pattern))
                        if rating_elem:
                            match = re.search(pattern, rating_elem)
                            if match:
                                rating = match.group()
                                break
                    
                    # Número de reseñas
                    num_reviews = "0"
                    review_patterns = [
                        r'(\d+)\s*(reseñas?|opiniones?|reviews?)',
                        r'(\d+)\s*total',
                        r'\((\d+)\)'
                    ]
                    
                    for pattern in review_patterns:
                        reviews_elem = card.find(text=re.compile(pattern, re.IGNORECASE))
                        if reviews_elem:
                            match = re.search(r'\d+', reviews_elem)
                            if match:
                                num_reviews = match.group()
                                break
                    
                    if company_name and company_name != "N/A":
                        companies.append({
                            'company_name': company_name,
                            'domain': domain,
                            'company_url': company_url,
                            'rating': rating,
                            'num_reviews': num_reviews,
                            'categories': 'travel_vacation'
                        })
                        
                except Exception as e:
                    print(f"Error al procesar tarjeta: {e}")
                    continue
        
        print(f"📊 Página {page}: {len(companies)} empresas acumuladas")
    
    # Eliminar duplicados
    seen = set()
    unique_companies = []
    for company in companies:
        if company['company_url'] not in seen:
            seen.add(company['company_url'])
            unique_companies.append(company)
    
    print(f"\n✅ Total de empresas únicas encontradas: {len(unique_companies)}")
    if unique_companies:
        print("📋 Primeras 3 empresas:")
        for i, company in enumerate(unique_companies[:3]):
            print(f"   {i+1}. {company['company_name']} - {company['rating']} ({company['num_reviews']} reseñas)")
    
    return unique_companies


In [ ]:
'''
# TEST: Ejecutar debugging para ver qué está pasando
driver = setup_driver(headless=False)  # Usar headless=True si no quieres ver el navegador
try:
    # Analizar la página de categoría
    category_url = "https://es.trustpilot.com/categories/travel_vacation"
    debug_page_structure(driver, category_url)
    
    print("\n" + "="*50)
    print("🔍 Intentando acceso directo a una empresa conocida...")
    
    # Intentar acceder directamente a una empresa conocida
    test_company_url = "https://es.trustpilot.com/review/www.booking.com"
    driver.get(test_company_url)
    time.sleep(3)
    
    if "booking" in driver.current_url.lower():
        print("✅ Acceso exitoso a página de empresa")
    else:
        print("❌ Redirigido o bloqueado")
        print(f"URL actual: {driver.current_url}")
        
finally:
    driver.quit()
    print("\n🔚 Test completado")
'''

In [8]:
# Función para extraer reseñas de una empresa con paginación
def get_reviews_from_company(driver, company_info, max_review_pages=3):
    """Extrae reseñas de una empresa específica con paginación"""
    reviews = []
    subcategories = ""  # Variable para almacenar las subcategorías
    
    for page in range(1, max_review_pages + 1):
        # Construir URL con paginación
        if page == 1:
            review_url = company_info['company_url']
        else:
            review_url = f"{company_info['company_url']}?page={page}"
        
        print(f"   📄 Página {page} de reseñas: {review_url}")
        driver.get(review_url)
        random_delay(2, 4)  # Delay aleatorio más humano
        
        # Extraer subcategorías del breadcrumb solo en la primera página
        if page == 1:
            try:
                soup_page = BeautifulSoup(driver.page_source, 'html.parser')
                
                # Buscar el breadcrumb - generalmente en un nav o ol/ul con clase que contiene 'breadcrumb'
                breadcrumb_selectors = [
                    {'tag': 'nav', 'attrs': {'aria-label': re.compile('breadcrumb', re.I)}},
                    {'tag': 'ol', 'attrs': {'class': re.compile('breadcrumb', re.I)}},
                    {'tag': 'ul', 'attrs': {'class': re.compile('breadcrumb', re.I)}},
                    {'tag': 'div', 'attrs': {'class': re.compile('breadcrumb', re.I)}},
                    # Selector específico de Trustpilot
                    {'tag': 'nav', 'attrs': {'class': re.compile('styles_breadcrumbs')}},
                ]
                
                breadcrumb_elem = None
                for selector in breadcrumb_selectors:
                    breadcrumb_elem = soup_page.find(selector['tag'], attrs=selector['attrs'])
                    if breadcrumb_elem:
                        break
                
                if breadcrumb_elem:
                    # Extraer todos los enlaces del breadcrumb
                    breadcrumb_links = breadcrumb_elem.find_all('a')
                    subcategory_list = []
                    
                    for link in breadcrumb_links:
                        text = link.text.strip()
                        # Ignorar elementos genéricos como "Home", "Inicio" o "Trustpilot"
                        if text and text.lower() not in ['home', 'inicio', 'trustpilot']:
                            subcategory_list.append(text)
                    
                    # Unir las subcategorías con " > "
                    subcategories = " > ".join(subcategory_list)
                    print(f"   📁 Subcategorías encontradas: {subcategories}")
                else:
                    print(f"   ⚠️ No se encontraron subcategorías")
                    
            except Exception as e:
                print(f"   ❌ Error al extraer subcategorías: {e}")
                subcategories = ""
        
        # Hacer scroll para cargar más reseñas en la página actual
        scroll_to_load_reviews(driver, max_scrolls=3)
        
        # Obtener HTML actualizado
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        # Buscar reseñas
        review_cards = soup.find_all('article', class_=re.compile('paper_paper__'))
        
        if not review_cards:
            print(f"   ⚠️ No se encontraron reseñas en la página {page}")
            break
        
        page_reviews = 0
        for card in review_cards:
            try:
                # Nombre del cliente
                customer_elem = card.find('span', attrs={'data-consumer-name-typography': 'true'})
                customer_name = customer_elem.text.strip() if customer_elem else "Anónimo"
                
                # Fecha de la reseña
                date_elem = card.find('time')
                review_date = date_elem.get('datetime', '') if date_elem else ""
                if not review_date:
                    date_text = date_elem.text.strip() if date_elem else ""
                    review_date = parse_date(date_text)
                
                # Puntuación (estrellas)
                rating_elem = card.find('div', attrs={'data-service-review-rating': True})
                if rating_elem:
                    rating_attr = rating_elem.get('data-service-review-rating', '0')
                    customer_score = int(rating_attr) if rating_attr.isdigit() else 0
                else:
                    # Buscar alternativa
                    star_elem = card.find('img', alt=re.compile('Valorado con'))
                    if star_elem:
                        alt_text = star_elem.get('alt', '')
                        score_match = re.search(r'(\d+)', alt_text)
                        customer_score = int(score_match.group(1)) if score_match else 0
                    else:
                        customer_score = 0
                
                # Texto de la reseña
                review_elem = card.find('p', attrs={'data-service-review-text-typography': 'true'})
                review_text = review_elem.text.strip() if review_elem else ""
                
                # Generar ID único
                review_id = generate_review_id(
                    company_info['company_name'], 
                    str(review_date), 
                    customer_name, 
                    review_text
                )
                
                reviews.append({
                    'review_id': review_id,
                    'domain': company_info['domain'],
                    'company_name': company_info['company_name'],
                    'categories': company_info['categories'],
                    'subcategories': subcategories,  # Nueva columna de subcategorías
                    'company_rating': company_info['rating'],
                    'review_date': review_date,
                    'customer_name': customer_name,
                    'customer_score': customer_score,
                    'review_text': review_text,
                    # Columnas vacías para análisis LLM
                    'language': '',
                    'sentiment': '',
                    'emotion': '',
                    'customer_gender': '',
                    'main_topic': '',
                    'keywords': '',
                    'customer_type': '',
                    'tourist_type': '',
                    'group_type': '',
                    'analyzed': False
                })
                page_reviews += 1
                
            except Exception as e:
                print(f"   ❌ Error al procesar reseña: {e}")
                continue
        
        print(f"   ✅ Página {page}: {page_reviews} reseñas extraídas")
        
        # Si no hay más páginas, salir del loop
        if page_reviews == 0:
            break
            
        # Verificar si hay botón "Siguiente" para continuar
        try:
            next_button = soup.find('a', {'aria-label': re.compile('Next|Siguiente', re.I)})
            if not next_button or 'disabled' in str(next_button.get('class', [])):
                print(f"   🏁 No hay más páginas de reseñas")
                break
        except:
            pass
    
    print(f"📊 Total: {len(reviews)} reseñas extraídas de {company_info['company_name']}")
    return reviews


In [9]:
# Función principal del scraper
def scrape_trustpilot_travel(max_companies=10, max_review_pages_per_company=3, max_company_pages=3):
    """
    Función principal que ejecuta todo el proceso de scraping
    
    Parámetros:
    - max_companies: Número máximo de empresas a procesar
    - max_review_pages_per_company: Número máximo de páginas de reseñas por empresa
    - max_company_pages: Número máximo de páginas de la categoría a recorrer
    """
    # URL de la categoría de viajes y vacaciones
    category_url = "https://es.trustpilot.com/categories/travel_vacation"
    
    # Inicializar driver
    print("🚀 Iniciando navegador...")
    driver = setup_driver()
    
    try:
        # Obtener lista de empresas
        print(f"\n🔍 Buscando empresas en la categoría de viajes...")
        print(f"   • Páginas de categoría a recorrer: {max_company_pages}")
        companies = get_companies_from_category(driver, category_url, max_pages=max_company_pages)
        
        # Limitar número de empresas
        companies = companies[:max_companies]
        print(f"\n📋 Total de empresas a procesar: {len(companies)}")
        print(f"   • Páginas de reseñas por empresa: {max_review_pages_per_company}")
        
        # Extraer reseñas de cada empresa
        all_reviews = []
        
        for i, company in enumerate(tqdm(companies, desc="Procesando empresas")):
            print(f"\n[{i+1}/{len(companies)}] 🏢 Procesando: {company['company_name']}")
            
            try:
                reviews = get_reviews_from_company(driver, company, max_review_pages=max_review_pages_per_company)
                all_reviews.extend(reviews)
                
                # Pausa entre empresas para evitar bloqueos
                random_delay(2, 4)
                
            except Exception as e:
                print(f"❌ Error al procesar {company['company_name']}: {e}")
                continue
        
        # Crear DataFrame
        df_reviews = pd.DataFrame(all_reviews)
        
        # Guardar resultados
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"trustpilot_travel_reviews_{timestamp}.csv"
        df_reviews.to_csv(filename, index=False, encoding='utf-8-sig')
        
        print(f"\n✅ Scraping completado!")
        print(f"📊 Total de reseñas extraídas: {len(all_reviews)}")
        print(f"💾 Archivo guardado: {filename}")
        
        # Mostrar estadísticas
        if len(df_reviews) > 0:
            print(f"\n📈 Estadísticas:")
            print(f"   • Empresas únicas: {df_reviews['company_name'].nunique()}")
            print(f"   • Promedio de reseñas por empresa: {len(df_reviews) / df_reviews['company_name'].nunique():.1f}")
            print(f"   • Distribución de puntuaciones:")
            score_dist = df_reviews['customer_score'].value_counts().sort_index()
            for score, count in score_dist.items():
                print(f"     ⭐ {score}: {count} reseñas ({count/len(df_reviews)*100:.1f}%)")
        
        return df_reviews
        
    finally:
        # Cerrar navegador
        driver.quit()
        print("\n🔚 Navegador cerrado.")


In [ ]:
# ❌ SCRAPER VIEJO SIN CHECKPOINTS - NO USAR
# (Este scraper no tiene sistema de checkpoints, usa el nuevo con checkpoints)
# df_results = scrape_trustpilot_travel(
#     max_companies=1500,                    # Número de empresas a procesar
#     max_review_pages_per_company=150,     # Páginas de reseñas por empresa (cada página tiene ~20 reseñas)
#     max_company_pages=75                 # Páginas de la categoría a recorrer (cada página tiene ~10 empresas)
# )


In [ ]:
# ❌ ESTADÍSTICAS DEL SCRAPER VIEJO - NO USAR
# (Depende del scraper viejo, usa las funciones de consolidación nuevas)
# if 'df_results' in locals():
#     print("📊 Resumen del dataset:")
#     print(f"Total de reseñas: {len(df_results)}")
#     print(f"Empresas únicas: {df_results['company_name'].nunique()}")
#     print(f"\nDistribución de puntuaciones:")
#     print(df_results['customer_score'].value_counts().sort_index())
#     
#     # Mostrar primeras filas
#     print("\n📋 Primeras 5 reseñas:")
#     display(df_results.head())


In [12]:
# Función para cargar y combinar múltiples archivos CSV
def combine_csv_files(pattern="trustpilot_travel_reviews_*.csv"):
    """Combina múltiples archivos CSV en un solo DataFrame"""
    import glob
    
    files = glob.glob(pattern)
    if not files:
        print("No se encontraron archivos CSV")
        return None
    
    dfs = []
    for file in files:
        df = pd.read_csv(file, encoding='utf-8-sig')
        dfs.append(df)
    
    combined_df = pd.concat(dfs, ignore_index=True)
    
    # Eliminar duplicados basados en review_id
    combined_df = combined_df.drop_duplicates(subset=['review_id'])
    
    print(f"Archivos combinados: {len(files)}")
    print(f"Total de reseñas únicas: {len(combined_df)}")
    
    return combined_df


In [10]:
# Sistema de Checkpoints mejorado - Usando CSV
import os
import glob
from datetime import datetime

class ScraperCheckpointCSV:
    """Sistema de checkpoints usando archivos CSV para el scraper de Trustpilot"""
    
    def __init__(self, session_id=None):
        # Crear ID de sesión único si no se proporciona
        if session_id is None:
            session_id = datetime.now().strftime("%Y%m%d_%H%M%S")
        
        self.session_id = session_id
        self.checkpoint_dir = f"checkpoints/{session_id}"
        self.progress_file = f"{self.checkpoint_dir}/progress.csv"
        self.errors_file = f"{self.checkpoint_dir}/errors.csv"
        self.companies_file = f"{self.checkpoint_dir}/companies_processed.csv"
        
        # Crear directorio de checkpoints
        os.makedirs(self.checkpoint_dir, exist_ok=True)
        
        print(f"📁 Sesión de checkpoint: {session_id}")
        print(f"📁 Directorio: {self.checkpoint_dir}")
        
        # Inicializar archivos si no existen
        self._init_checkpoint_files()
    
    def _init_checkpoint_files(self):
        """Inicializa los archivos CSV de checkpoint si no existen"""
        
        # Archivo de progreso general
        if not os.path.exists(self.progress_file):
            progress_df = pd.DataFrame([{
                'session_id': self.session_id,
                'start_time': datetime.now().isoformat(),
                'last_update': datetime.now().isoformat(),
                'status': 'iniciado',
                'companies_processed': 0,
                'total_reviews': 0,
                'total_csv_files': 0
            }])
            progress_df.to_csv(self.progress_file, index=False)
        
        # Archivo de empresas procesadas
        if not os.path.exists(self.companies_file):
            companies_df = pd.DataFrame(columns=[
                'company_url', 'company_name', 'domain', 'processed_at', 
                'reviews_count', 'csv_filename', 'status'
            ])
            companies_df.to_csv(self.companies_file, index=False)
        
        # Archivo de errores
        if not os.path.exists(self.errors_file):
            errors_df = pd.DataFrame(columns=[
                'timestamp', 'company_name', 'company_url', 'error_type', 'error_message'
            ])
            errors_df.to_csv(self.errors_file, index=False)
    
    def add_processed_company(self, company_info, reviews_count, csv_filename):
        """Registra una empresa como procesada en el checkpoint CSV"""
        
        # Actualizar archivo de empresas procesadas
        new_row = pd.DataFrame([{
            'company_url': company_info['company_url'],
            'company_name': company_info['company_name'],
            'domain': company_info['domain'],
            'processed_at': datetime.now().isoformat(),
            'reviews_count': reviews_count,
            'csv_filename': csv_filename,
            'status': 'completado'
        }])
        
        # Leer archivo existente y agregar nueva fila
        if os.path.exists(self.companies_file):
            companies_df = pd.read_csv(self.companies_file)
            companies_df = pd.concat([companies_df, new_row], ignore_index=True)
        else:
            companies_df = new_row
        
        companies_df.to_csv(self.companies_file, index=False)
        
        # Actualizar progreso general
        self._update_progress()
        
        print(f"   📝 Checkpoint actualizado: {company_info['company_name']} - {reviews_count} reseñas")
    
    def _update_progress(self):
        """Actualiza el archivo de progreso general"""
        # Leer empresas procesadas
        if os.path.exists(self.companies_file):
            companies_df = pd.read_csv(self.companies_file)
            total_companies = len(companies_df)
            total_reviews = companies_df['reviews_count'].sum()
        else:
            total_companies = 0
            total_reviews = 0
        
        # Contar archivos CSV generados
        csv_files = glob.glob(f"{self.checkpoint_dir}/reviews_*.csv")
        total_csv_files = len(csv_files)
        
        # Actualizar progreso
        progress_data = {
            'session_id': self.session_id,
            'start_time': self._get_start_time(),
            'last_update': datetime.now().isoformat(),
            'status': 'en_progreso',
            'companies_processed': total_companies,
            'total_reviews': total_reviews,
            'total_csv_files': total_csv_files
        }
        
        progress_df = pd.DataFrame([progress_data])
        progress_df.to_csv(self.progress_file, index=False)
    
    def _get_start_time(self):
        """Obtiene la hora de inicio de la sesión"""
        if os.path.exists(self.progress_file):
            progress_df = pd.read_csv(self.progress_file)
            return progress_df.iloc[0]['start_time']
        return datetime.now().isoformat()
    
    def is_company_processed(self, company_url):
        """Verifica si una empresa ya fue procesada"""
        if not os.path.exists(self.companies_file):
            return False
        
        companies_df = pd.read_csv(self.companies_file)
        return company_url in companies_df['company_url'].values
    
    def log_error(self, company_name, company_url, error_type, error_message):
        """Registra un error en el checkpoint CSV"""
        new_error = pd.DataFrame([{
            'timestamp': datetime.now().isoformat(),
            'company_name': company_name,
            'company_url': company_url,
            'error_type': error_type,
            'error_message': str(error_message)
        }])
        
        if os.path.exists(self.errors_file):
            errors_df = pd.read_csv(self.errors_file)
            errors_df = pd.concat([errors_df, new_error], ignore_index=True)
        else:
            errors_df = new_error
        
        errors_df.to_csv(self.errors_file, index=False)
    
    def get_processed_companies(self):
        """Obtiene la lista de empresas ya procesadas"""
        if not os.path.exists(self.companies_file):
            return []
        
        companies_df = pd.read_csv(self.companies_file)
        return companies_df['company_url'].tolist()
    
    def get_summary(self):
        """Obtiene un resumen del estado actual"""
        if not os.path.exists(self.progress_file):
            return {"estado": "sin_checkpoint"}
        
        progress_df = pd.read_csv(self.progress_file)
        progress = progress_df.iloc[0]
        
        summary = {
            'Sesión': self.session_id,
            'Estado': progress['status'],
            'Inicio': progress['start_time'],
            'Última actualización': progress['last_update'],
            'Empresas procesadas': progress['companies_processed'],
            'Total de reseñas': progress['total_reviews'],
            'Archivos CSV': progress['total_csv_files']
        }
        
        # Agregar info de errores si existen
        if os.path.exists(self.errors_file):
            errors_df = pd.read_csv(self.errors_file)
            summary['Errores registrados'] = len(errors_df)
        
        return summary
    
    def save_reviews_csv(self, company_info, reviews):
        """Guarda las reseñas en un archivo CSV individual"""
        if not reviews:
            return None
        
        # Crear nombre de archivo único
        timestamp = datetime.now().strftime("%H%M%S")
        csv_filename = f"{self.checkpoint_dir}/reviews_{company_info['domain']}_{timestamp}.csv"
        
        # Convertir a DataFrame y guardar
        df_reviews = pd.DataFrame(reviews)
        df_reviews.to_csv(csv_filename, index=False, encoding='utf-8-sig')
        
        # Solo devolver el nombre del archivo, no la ruta completa
        return f"reviews_{company_info['domain']}_{timestamp}.csv"
    
    def consolidate_all_reviews(self):
        """Consolida todos los archivos CSV de reseñas en uno solo"""
        csv_pattern = f"{self.checkpoint_dir}/reviews_*.csv"
        csv_files = glob.glob(csv_pattern)
        
        if not csv_files:
            print("📭 No se encontraron archivos CSV de reseñas para consolidar")
            return None
        
        print(f"📁 Consolidando {len(csv_files)} archivos CSV...")
        
        all_reviews = []
        for csv_file in csv_files:
            try:
                df_temp = pd.read_csv(csv_file, encoding='utf-8-sig')
                all_reviews.append(df_temp)
                print(f"   ✅ {os.path.basename(csv_file)}: {len(df_temp)} reseñas")
            except Exception as e:
                print(f"   ❌ Error leyendo {csv_file}: {e}")
        
        if all_reviews:
            # Consolidar todos los DataFrames
            df_consolidated = pd.concat(all_reviews, ignore_index=True)
            
            # Eliminar duplicados por review_id
            initial_count = len(df_consolidated)
            df_consolidated = df_consolidated.drop_duplicates(subset=['review_id'])
            final_count = len(df_consolidated)
            
            # Guardar archivo consolidado
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            consolidated_filename = f"trustpilot_consolidated_{self.session_id}_{timestamp}.csv"
            df_consolidated.to_csv(consolidated_filename, index=False, encoding='utf-8-sig')
            
            # Actualizar estado a completado
            progress_data = {
                'session_id': self.session_id,
                'start_time': self._get_start_time(),
                'last_update': datetime.now().isoformat(),
                'status': 'completado',
                'companies_processed': len(pd.read_csv(self.companies_file)) if os.path.exists(self.companies_file) else 0,
                'total_reviews': final_count,
                'total_csv_files': len(csv_files)
            }
            progress_df = pd.DataFrame([progress_data])
            progress_df.to_csv(self.progress_file, index=False)
            
            print(f"\n✅ Consolidación completada!")
            print(f"📊 Archivo final: {consolidated_filename}")
            print(f"📊 Total de reseñas: {final_count}")
            print(f"📊 Duplicados eliminados: {initial_count - final_count}")
            print(f"🏢 Empresas únicas: {df_consolidated['company_name'].nunique()}")
            
            return df_consolidated
        
        return None


In [11]:
# Función principal mejorada con sistema de checkpoints CSV
def scrape_trustpilot_travel_with_checkpoint(max_companies=10, 
                                            max_review_pages_per_company=3, 
                                            max_company_pages=3,
                                            session_id=None,
                                            resume=True):
    """
    Función principal con sistema de checkpoints CSV - TOTALMENTE AUTOMÁTICA
    
    Parámetros:
    - max_companies: Número máximo de empresas a procesar
    - max_review_pages_per_company: Páginas de reseñas por empresa
    - max_company_pages: Páginas de categoría a recorrer
    - session_id: ID de sesión específico (opcional)
    - resume: Si True, intenta resumir desde el último checkpoint
    """
    
    # 🤖 LÓGICA AUTOMÁTICA DE DETECCIÓN DE CHECKPOINTS
    if session_id is None and resume:
        # Buscar si hay sesiones existentes
        if os.path.exists("checkpoints"):
            sessions = [d for d in os.listdir("checkpoints") if os.path.isdir(f"checkpoints/{d}")]
            if sessions:
                # Usar la sesión más reciente automáticamente
                latest_session = sorted(sessions)[-1]
                print(f"🔄 MODO AUTOMÁTICO: Detectada sesión existente {latest_session}")
                print(f"♻️ Reanudando automáticamente desde el último checkpoint...")
                session_id = latest_session
            else:
                print(f"🆕 MODO AUTOMÁTICO: No hay sesiones previas, iniciando nueva sesión...")
        else:
            print(f"🆕 MODO AUTOMÁTICO: No hay checkpoints, iniciando desde cero...")
    
    # Inicializar sistema de checkpoints CSV
    checkpoint = ScraperCheckpointCSV(session_id=session_id)
    
    # URL de la categoría
    category_url = "https://es.trustpilot.com/categories/travel_vacation"
    
    # Inicializar driver
    print("🚀 Iniciando navegador...")
    driver = setup_driver()
    
    try:
        # Obtener lista de empresas
        print(f"\n🔍 Buscando empresas en la categoría de viajes...")
        companies = get_companies_from_category(driver, category_url, max_pages=max_company_pages)
        
        # Filtrar empresas ya procesadas si estamos resumiendo
        if resume:
            processed_urls = checkpoint.get_processed_companies()
            if processed_urls:
                print(f"\n♻️ Filtrando empresas ya procesadas...")
                companies_pendientes = [c for c in companies if c['company_url'] not in processed_urls]
                print(f"   • Empresas totales: {len(companies)}")
                print(f"   • Ya procesadas: {len(processed_urls)}")
                print(f"   • Pendientes: {len(companies_pendientes)}")
                companies = companies_pendientes
        
        # Limitar número de empresas
        companies = companies[:max_companies]
        print(f"\n📋 Total de empresas a procesar: {len(companies)}")
        
        # Procesar empresas
        for i, company in enumerate(companies):
            print(f"\n[{i+1}/{len(companies)}] 🏢 Procesando: {company['company_name']}")
            
            try:
                # Extraer reseñas
                reviews = get_reviews_from_company(driver, company, max_review_pages=max_review_pages_per_company)
                
                if reviews:
                    # Guardar reseñas en CSV individual
                    csv_filename = checkpoint.save_reviews_csv(company, reviews)
                    
                    # Actualizar checkpoint
                    checkpoint.add_processed_company(company, len(reviews), csv_filename)
                    print(f"   ✅ {len(reviews)} reseñas guardadas en {csv_filename}")
                else:
                    print(f"   ⚠️ No se encontraron reseñas para {company['company_name']}")
                    # Registrar empresa sin reseñas
                    checkpoint.add_processed_company(company, 0, None)
                
                # Pausa entre empresas
                random_delay(2, 4)
                
            except Exception as e:
                print(f"   ❌ Error al procesar {company['company_name']}: {e}")
                checkpoint.log_error(company['company_name'], company['company_url'], 'scraping_error', str(e))
                continue
        
        print(f"\n🎯 Procesamiento completado!")
        
        # Mostrar resumen
        summary = checkpoint.get_summary()
        print("\n📈 Resumen de la sesión:")
        for key, value in summary.items():
            print(f"   • {key}: {value}")
            
        return checkpoint
            
    except KeyboardInterrupt:
        print("\n⚠️ Proceso interrumpido por el usuario")
        print("💾 El progreso se ha guardado. Puedes resumir la ejecución más tarde.")
        print(f"💾 ID de sesión: {checkpoint.session_id}")
        return checkpoint
        
    except Exception as e:
        print(f"\n❌ Error general: {e}")
        checkpoint.log_error('SISTEMA', 'GENERAL', 'sistema_error', str(e))
        return checkpoint
        
    finally:
        # Cerrar navegador
        driver.quit()
        print("\n🔚 Navegador cerrado.")


In [12]:
# Funciones de utilidad para gestión de checkpoints CSV

def list_checkpoint_sessions():
    """Lista todas las sesiones de checkpoint disponibles"""
    if not os.path.exists("checkpoints"):
        print("📭 No hay sesiones de checkpoint")
        return []
    
    sessions = [d for d in os.listdir("checkpoints") if os.path.isdir(f"checkpoints/{d}")]
    
    if not sessions:
        print("📭 No hay sesiones de checkpoint")
        return []
    
    print("📁 Sesiones disponibles:")
    session_data = []
    
    for session_id in sorted(sessions):
        try:
            checkpoint = ScraperCheckpointCSV(session_id=session_id)
            summary = checkpoint.get_summary()
            session_data.append({
                'session_id': session_id,
                'status': summary.get('Estado', 'desconocido'),
                'companies': summary.get('Empresas procesadas', 0),
                'reviews': summary.get('Total de reseñas', 0),
                'last_update': summary.get('Última actualización', 'N/A')
            })
            print(f"   🔹 {session_id}: {summary.get('Estado')} - {summary.get('Empresas procesadas', 0)} empresas, {summary.get('Total de reseñas', 0)} reseñas")
        except:
            print(f"   ❌ {session_id}: Error al leer sesión")
    
    return session_data

def view_checkpoint_status(session_id=None):
    """Muestra el estado de una sesión específica o la más reciente"""
    if session_id is None:
        # Buscar la sesión más reciente
        sessions = list_checkpoint_sessions()
        if not sessions:
            return
        session_id = sessions[-1]['session_id']
        print(f"\n📊 Mostrando sesión más reciente: {session_id}")
    
    try:
        checkpoint = ScraperCheckpointCSV(session_id=session_id)
        summary = checkpoint.get_summary()
        
        print("\n📊 ESTADO DEL CHECKPOINT")
        print("=" * 50)
        for key, value in summary.items():
            print(f"{key}: {value}")
        
        # Mostrar empresas procesadas
        if os.path.exists(checkpoint.companies_file):
            companies_df = pd.read_csv(checkpoint.companies_file)
            if len(companies_df) > 0:
                print(f"\n📋 Últimas 5 empresas procesadas:")
                for _, company in companies_df.tail(5).iterrows():
                    status_icon = "✅" if company['status'] == 'completado' else "⚠️"
                    print(f"   {status_icon} {company['company_name']}: {company['reviews_count']} reseñas")
        
        # Mostrar errores si existen
        if os.path.exists(checkpoint.errors_file):
            errors_df = pd.read_csv(checkpoint.errors_file)
            if len(errors_df) > 0:
                print(f"\n⚠️ Últimos 3 errores:")
                for _, error in errors_df.tail(3).iterrows():
                    print(f"   • {error['company_name']}: {error['error_message'][:50]}...")
    
    except Exception as e:
        print(f"❌ Error al leer sesión {session_id}: {e}")

def consolidate_session_reviews(session_id=None):
    """Consolida todas las reseñas de una sesión específica"""
    if session_id is None:
        # Usar la sesión más reciente
        sessions = list_checkpoint_sessions()
        if not sessions:
            print("📭 No hay sesiones disponibles")
            return None
        session_id = sessions[-1]['session_id']
        print(f"🔍 Usando sesión más reciente: {session_id}")
    
    try:
        checkpoint = ScraperCheckpointCSV(session_id=session_id)
        df_consolidated = checkpoint.consolidate_all_reviews()
        return df_consolidated
    
    except Exception as e:
        print(f"❌ Error al consolidar sesión {session_id}: {e}")
        return None

def resume_session(session_id, max_companies=None, max_review_pages_per_company=3, max_company_pages=3):
    """Reanuda una sesión específica de scraping"""
    print(f"♻️ Reanudando sesión: {session_id}")
    
    return scrape_trustpilot_travel_with_checkpoint(
        max_companies=max_companies or 9999,  # Si no se especifica, procesar todas las pendientes
        max_review_pages_per_company=max_review_pages_per_company,
        max_company_pages=max_company_pages,
        session_id=session_id,
        resume=True
    )

def export_session_errors(session_id=None):
    """Exporta los errores de una sesión específica"""
    if session_id is None:
        sessions = list_checkpoint_sessions()
        if not sessions:
            print("📭 No hay sesiones disponibles")
            return
        session_id = sessions[-1]['session_id']
    
    try:
        checkpoint = ScraperCheckpointCSV(session_id=session_id)
        
        if not os.path.exists(checkpoint.errors_file):
            print("✅ No hay errores registrados en esta sesión")
            return
        
        errors_df = pd.read_csv(checkpoint.errors_file)
        if len(errors_df) == 0:
            print("✅ No hay errores registrados en esta sesión")
            return
        
        # Guardar copia de errores con timestamp
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        output_filename = f"errores_{session_id}_{timestamp}.csv"
        errors_df.to_csv(output_filename, index=False)
        
        print(f"📋 Errores exportados: {output_filename}")
        print(f"   Total de errores: {len(errors_df)}")
        
        # Mostrar resumen de tipos de errores
        print(f"\n📊 Tipos de errores:")
        error_summary = errors_df['error_type'].value_counts()
        for error_type, count in error_summary.items():
            print(f"   • {error_type}: {count}")
    
    except Exception as e:
        print(f"❌ Error al exportar errores de sesión {session_id}: {e}")

def delete_session(session_id, confirm=True):
    """Elimina una sesión completa de checkpoint"""
    if confirm:
        response = input(f"⚠️ ¿Estás seguro de que quieres eliminar la sesión {session_id}? (s/n): ")
        if response.lower() != 's':
            print("❌ Operación cancelada")
            return
    
    import shutil
    session_dir = f"checkpoints/{session_id}"
    
    if os.path.exists(session_dir):
        shutil.rmtree(session_dir)
        print(f"🗑️ Sesión {session_id} eliminada")
    else:
        print(f"❌ La sesión {session_id} no existe")


In [ ]:
# 🚀 PASO 1: EJECUTAR SCRAPER CON CHECKPOINTS AUTOMÁTICOS
# Descomenta las líneas de abajo para iniciar (quita los # del inicio):

checkpoint = scrape_trustpilot_travel_with_checkpoint(
    max_companies=1500,                    # Procesar hasta 1500 empresas
    max_review_pages_per_company=150,      # 150 páginas de reseñas por empresa
    max_company_pages=75,                  # Buscar en 75 páginas de la categoría
    session_id=None,                       # Se creará ID automático si no hay sesiones
    resume=True                            # AUTOMÁTICO: reanuda si hay checkpoint
)


In [ ]:
# 📊 PASO 2: MONITOREO AUTOMÁTICO - Ver progreso en tiempo real
# Descomenta para ver el estado actual (quita los # del inicio):

list_checkpoint_sessions()  # Ver todas las sesiones
print("\n" + "="*50)
view_checkpoint_status()    # Ver detalles de la sesión más reciente


In [ ]:
# 🔍 OPCIONAL: Ver estado de una sesión específica
# view_checkpoint_status("20241201_143022")  # Para sesión específica (descomenta si necesitas)


In [ ]:
# ♻️ SOLO SI NECESITAS: Reanudar una sesión específica manualmente
# (El modo automático ya hace esto, pero si quieres forzar una sesión específica:)
# checkpoint = resume_session(
#     session_id="20241201_143022",         # ID de la sesión a reanudar
#     max_companies=100,                    # Continuar hasta 100 empresas en total
#     max_review_pages_per_company=3
# )


In [ ]:
# 📁 PASO 3: CONSOLIDACIÓN AUTOMÁTICA - Unir todos los CSV en un archivo final
# Descomenta para consolidar automáticamente (quita los # del inicio):

df_final = consolidate_session_reviews()  # Sesión más reciente automáticamente

# # Mostrar estadísticas del resultado final:
if df_final is not None:
    print(f"\n🎉 CONSOLIDACIÓN COMPLETADA!")
    print(f"📊 Total de reseñas consolidadas: {len(df_final):,}")
    print(f"🏢 Empresas únicas: {df_final['company_name'].nunique()}")
    print(f"📈 Promedio de reseñas por empresa: {len(df_final) / df_final['company_name'].nunique():.1f}")
    print("\n⭐ Distribución de puntuaciones:")
    score_dist = df_final['customer_score'].value_counts().sort_index()
    for score, count in score_dist.items():
        percentage = (count / len(df_final)) * 100
        print(f"   {score} estrellas: {count:,} reseñas ({percentage:.1f}%)")
else:
    print("⚠️ No hay datos para consolidar")


In [ ]:
# 📋 OPCIONAL: Gestión de errores y limpieza (descomenta solo si necesitas)
# export_session_errors()                  # Exportar errores de la sesión más reciente
# export_session_errors("20241201_143022") # Exportar errores de sesión específica
# delete_session("20241201_143022")        # Eliminar una sesión completa (con confirmación)


In [ ]:
'''
# Función para filtrar reseñas por puntuación o fecha
def filter_reviews(df, min_score=None, max_score=None, start_date=None, end_date=None):
    """Filtra reseñas según criterios específicos"""
    filtered_df = df.copy()
    
    if min_score is not None:
        filtered_df = filtered_df[filtered_df['customer_score'] >= min_score]
    
    if max_score is not None:
        filtered_df = filtered_df[filtered_df['customer_score'] <= max_score]
    
    if start_date is not None:
        filtered_df['review_date'] = pd.to_datetime(filtered_df['review_date'])
        filtered_df = filtered_df[filtered_df['review_date'] >= start_date]
    
    if end_date is not None:
        filtered_df['review_date'] = pd.to_datetime(filtered_df['review_date'])
        filtered_df = filtered_df[filtered_df['review_date'] <= end_date]
    
    print(f"Reseñas filtradas: {len(filtered_df)} de {len(df)}")
    return filtered_df

# Ejemplo de uso:
# df_filtered = filter_reviews(df_results, min_score=4, start_date='2024-01-01')
'''